# 0.0 Imports

In [16]:
import pandas as pd
import numpy as np
import inflection

# 0.1 Loading Data

In [17]:
df_ready = pd.read_csv('dataset/df_ready.csv')

# 1.0 Data Description

In [18]:
df = df_ready.copy()

## 1.1 Rename Columns

In [19]:
cols_old = ['Unnamed: 0', 'Date_imp', 'Date_imp_d', 'Cluster', 'Category_name',
            'name', 'price', 'disc_price', 'merchant', 'condition',
            'Disc_percentage', 'isSale', 'Imp_count', 'brand', 'p_description',
            'currency', 'dateAdded', 'dateSeen', 'dateUpdated', 'imageURLs',
            'manufacturer', 'shipping', 'sourceURLs', 'weight', 'Date_imp_d.1',
            'Day_n', 'month', 'month_n', 'day', 'Week_Number', 'Zscore_1', 'price_std']

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

df.columns = cols_new

## 1.2 Data Dimensions

In [20]:
print('Number of Rows: {}'.format(df.shape[0]))
print('Number of Cols: {}'.format(df.shape[1]))

Number of Rows: 23151
Number of Cols: 32


## 1.3 Data Types

In [21]:
df.dtypes

unnamed: 0           int64
date_imp            object
date_imp_d          object
cluster              int64
category_name       object
name                object
price              float64
disc_price         float64
merchant            object
condition           object
disc_percentage    float64
is_sale             object
imp_count            int64
brand               object
p_description       object
currency            object
date_added          object
date_seen           object
date_updated        object
image_ur_ls         object
manufacturer        object
shipping            object
source_ur_ls        object
weight              object
date_imp_d.1        object
day_n               object
month                int64
month_n             object
day                  int64
week_number          int64
zscore_1           float64
price_std          float64
dtype: object